In [2]:
import pandas as pd
import numpy as np

In [3]:
senate_cand = pd.read_excel("Senate_2018_data.xlsx")
print(senate_cand["Cand_Office_St"].value_counts().shape)
senate_cand.head()

(34,)


,Cand_Name,Cand_Id,Cand_Office,Cand_Office_St,Cand_Office_Dist,Cand_Party_Affiliation,Cand_Incumbent_Challenger_Open_Seat,Total_Receipt,Total_Disbursement,Cand_State,Individual_Contribution,Other_Committee_Contribution,Party_Committee_Contribution
0,"ABBOUD, DEEDRA",S8AZ00122,S,AZ,0,DEM,OPEN,113399.05,95070.97,AZ,94301.05,0.0,0.0
1,"BRITTAIN, CRAIG R",S8AZ00189,S,AZ,0,REP,OPEN,0.00,0.00,AZ,0.00,0.0,0.0
2,"KOKESH, ADAM",S8AZ00296,S,AZ,0,LIB,OPEN,0.00,0.00,AZ,0.00,0.0,0.0
3,"RUSSELL, DALE CHRISTOPHER",S8AZ00148,S,AZ,0,DEM,OPEN,13567.00,17887.00,AZ,13447.00,0.0,0.0
4,"MOSS, JIM",S8AZ00155,S,AZ,0,DEM,OPEN,32445.00,42071.00,AZ,2168.00,0.0,0.0


In [4]:
senate_results = pd.read_excel("Senate_2018_results.xlsx")
senate_results = senate_results.filter(["Cand_Id","GE WINNER INDICATOR"])
senate_results['GE WINNER INDICATOR'] = senate_results['GE WINNER INDICATOR'].fillna(0)
print(senate_results['GE WINNER INDICATOR'].value_counts())

senate_results.head()

0.0    449
1.0     33
Name: GE WINNER INDICATOR, dtype: int64


,Cand_Id,GE WINNER INDICATOR
0,S8AZ00122,0.0
1,S8AZ00189,0.0
2,S8AZ00296,0.0
3,S8AZ00148,0.0
4,S8AZ00155,0.0


In [5]:
senate_2018 = pd.merge(senate_cand,senate_results, how = 'left', left_on = 'Cand_Id', right_on = 'Cand_Id')
print(senate_2018['GE WINNER INDICATOR'].value_counts())
senate_2018.head()

0.0    449
1.0     33
Name: GE WINNER INDICATOR, dtype: int64


,Cand_Name,Cand_Id,Cand_Office,Cand_Office_St,Cand_Office_Dist,Cand_Party_Affiliation,Cand_Incumbent_Challenger_Open_Seat,Total_Receipt,Total_Disbursement,Cand_State,Individual_Contribution,Other_Committee_Contribution,Party_Committee_Contribution,GE WINNER INDICATOR
0,"ABBOUD, DEEDRA",S8AZ00122,S,AZ,0,DEM,OPEN,113399.05,95070.97,AZ,94301.05,0.0,0.0,0.0
1,"BRITTAIN, CRAIG R",S8AZ00189,S,AZ,0,REP,OPEN,0.00,0.00,AZ,0.00,0.0,0.0,0.0
2,"KOKESH, ADAM",S8AZ00296,S,AZ,0,LIB,OPEN,0.00,0.00,AZ,0.00,0.0,0.0,0.0
3,"RUSSELL, DALE CHRISTOPHER",S8AZ00148,S,AZ,0,DEM,OPEN,13567.00,17887.00,AZ,13447.00,0.0,0.0,0.0
4,"MOSS, JIM",S8AZ00155,S,AZ,0,DEM,OPEN,32445.00,42071.00,AZ,2168.00,0.0,0.0,0.0


In [6]:
senate_2018.to_csv("test_dataset.csv",index=False)

In [7]:
def generate_max_feature(df,col_name,name_of_feature,groupby_params=["Cand_Office_St"]):
    '''""
    Generate a feature that will groupby state, district and get the top candidate for the given feature
    
    For eg: Individual_Contribution
    The function will identify candidates that have the highest in "Individual_Contribution"
    in their district,state and assign 1.
    Rest all will have 0.
    '''
    max_rows = df.groupby(groupby_params)[col_name].idxmax()
    df[name_of_feature] = 0
    df.loc[max_rows,name_of_feature] = 1
    return df

In [8]:
senate_2018 = generate_max_feature(senate_2018,"Individual_Contribution","top_individual_contribution")
senate_2018 = generate_max_feature(senate_2018,"Total_Disbursement","top_total_disbursement")
senate_2018 = generate_max_feature(senate_2018,"Other_Committee_Contribution","top_other_comm_contribution")
senate_2018 = generate_max_feature(senate_2018,"Party_Committee_Contribution","top_party_comm_contribution",["Cand_Office_St","Cand_Party_Affiliation"])

In [9]:
print("No. of winners = " + str(senate_2018[senate_2018["GE WINNER INDICATOR"] == 1].shape[0]))

print("No. of unique states = " + str(senate_2018["Cand_Office_St"].unique().shape[0]))

No. of winners = 33
No. of unique states = 34


In [10]:
# In-state out-state candidates. Currently we are dropping with Nan values in Cand_State
senate_2018["in_state"] = 1
senate_2018.loc[senate_2018["Cand_Office_St"] != senate_2018["Cand_State"],"in_state"] = 0

In [11]:
# Creating dummy variables for incumbent and open candidates
senate_2018["incumbent"] = 0
senate_2018.loc[senate_2018["Cand_Incumbent_Challenger_Open_Seat"] == "INCUMBENT","incumbent"] = 1

senate_2018["open"] = 0
senate_2018.loc[senate_2018["Cand_Incumbent_Challenger_Open_Seat"] == "OPEN","open"] = 1

In [15]:
senate_analytical = pd.read_csv("senate_analytical.csv")

senate_analytical["strong_DEM"].value_counts()

0    2043
1     199
Name: strong_DEM, dtype: int64

In [ ]:
senate_2018.loc[(senate_2018["Cand_Party_Affiliation"] != "DEM") & (senate_2018["Cand_Party_Affiliation"] != "REP"),"Cand_Party_Affiliation"] = "OTHER"
senate_2018 = pd.get_dummies(senate_2018,columns=["Cand_Party_Affiliation"])

In [ ]:
all_states = ['AK', 'AL', 'AR', 'CO', 'DE', 'GA', 'IA', 'ID', 'IL', 'KS', 'KY', 'LA', 'MA', 'ME', 'MI', 'MN', 'MS', 'MT', 'NC', 'NE', 'NH', 'NJ', 'NM', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'VA', 'WV', 'WY', 'AZ', 'CA', 'CT', 'FL', 'HI', 'IN', 'MD', 'MO', 'ND', 'NV', 'NY', 'OH', 'UT', 'VT', 'WA', 'WI']
states_in_data = senate_2018["Cand_Office_St"].unique().tolist()

states_series = senate_2018["Cand_Office_St"]

missing_states = set(all_states) - set(states_in_data)
print(missing_states)
senate_2018 = pd.get_dummies(senate_2018,columns=["Cand_Office_St"])

for state in missing_states:
    senate_2018["Cand_Office_St_" + state] = 0

senate_2018["Cand_Office_St"] = states_series
senate_2018.shape

In [ ]:
senate_2018.to_csv("test_analytical.csv",index=False)